# Finetuning a local LLM to autograde student essays

This notebook downloads pretrained GPT-2, runs the model locally, and finetunes the model on mock data for essay autograding.

### Download and import libraries

In [1]:
# Download libraries
%pip install transformers
%pip install transformers[torch]

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\irish\appdata\local\programs\python\python311\lib\site-packages\filelock-3.13.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\irish\appdata\local\programs\python\python311\lib\site-packages\fsspec-2023.10.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\irish\appdata\local\programs\python\python311\lib\site-packages\mpmath-1.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\irish

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\irish\appdata\local\programs\python\python311\lib\site-packages\filelock-3.13.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\irish\appdata\local\programs\python\python311\lib\site-packages\fsspec-2023.10.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\irish\appdata\local\programs\python\python311\lib\site-packages\mpmath-1.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\irish

In [2]:
# Import the needed libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

### Download model

In [3]:
# Load a pre-trained LLM and tokenizer
model_name = "gpt2" # Replace with name of the LLM you want to run
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name)

### Choose device for model

In [4]:
# Define the device to run the model (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Create dataset for finetuning

This generates mock data for a dataset of essays on which to fine-tune a local model based on the [ASAP dataset](https://www.kaggle.com/competitions/asap-aes/rules). Essays are generated with Google Gemini, and essay scores/targets are manually graded according to the ASAP dataset rubrics.

In [5]:
columns = ['essay_set', 'essay', 'ground_truth_score']
data = [
    [
        1,
        "Is everyone freaking out about computers for no reason? Computers are awesome! They're like having a superhero sidekick in your pocket. They can teach you anything you want to know, from how to build a volcano (for science class, of course!) to what life is like on the other side of the world. Plus, games help your hand-eye coordination, which is basically a superpower for gamers like me.\n\nSure, maybe some grown-ups spend too much time staring at screens, but that's their problem, not ours. We can totally use computers for good stuff and still hang out with friends or play outside.  So, lighten up everyone, computers are here to stay, and they're making our lives way cooler!\n\nYour friend,\n\nGamer Girl extraordinaire",
        2
    ],
    [
        1,
        "Dear Editor,\n\nComputers are like a double-edged sword. On one hand, they're amazing tools for learning and connecting. I can chat with my cousin in California even though she's miles away, and research projects are a breeze with all the information online. Plus, there are these awesome educational games that make learning history or even another language actually fun!\n\nBut on the other hand, sometimes computers can be a total time suck. You start checking one thing, and then BAM! Two hours later, you're watching cat videos (don't judge, they're hilarious!).  This can lead to neglecting real-life stuff like hanging out with friends or getting enough exercise.\n\nSo, I think the key is balance. Computers are great, but we gotta make sure they don't take over our lives. Maybe parents can set screen time limits, and we can all make a conscious effort to spend more time outside and with loved ones.\n\nSincerely,\n\nA Balanced Computer User",
        4
    ],
    [
        1,
        "Dear Editor,\n\nSome people say computers are turning us all into couch potatoes. They might have a point. It's way easier to watch funny videos online than go for a bike ride.  But computers can actually help you appreciate nature too! You can research cool animals or amazing places you want to visit someday.  Plus, there are apps that help you identify plants and track your hikes.\n\nThink of it like this: computers are like a superhighway to information, but sometimes you gotta take an exit and explore the real world. We can use computers to learn about nature, then go outside and experience it for ourselves. It's all about finding the right balance!\n\nYours truly,\n\nThe Tech-Savvy Nature Enthusiast",
        3
    ],
    [
        1,
        "Dear Editor,\n\nComputers aren't just about games and social media, you know! They can be a real spark for creativity.  I can write stories on the computer, way easier than using pen and paper. Plus, there are these amazing programs that let you design animations or even create your own music!  Imagine composing a song as cool as your favourite band, all on your computer.\n\nSure, some folks might say computers make us lazy, but I think they actually help us think outside the box. You can find inspiration for anything online, from historical fashion to crazy science experiments.  Computers are like a giant toolbox full of cool stuff to help you express yourself.\n\nSo next time someone tells you to get off the computer, tell them you're busy building your own creativity empire!\n\nYours creatively,\n\nThe Techie Da Vinci",
        4
    ],
    [
        1,
        "Dear Editor,\n\nLet's be honest, computers can be way better friends than some actual people. They're always there, 24/7, whenever you need them. Feeling bored? Boom, there's a million games to play. Need help with homework? Online resources can explain anything. Plus, you can connect with people who share your interests, no matter how weird or wonderful they are.\n\nOf course, computers can't replace real-life friends entirely. But they can be a great way to meet new people, especially if you're shy or new to town.  Think of them as awesome companions who can help you learn, play, and connect with the world.\n\nYour friend (both online and off),\n\nThe Social Butterfly with a Keyboard",
        3
    ],
    [
        1,
        "Dear Editor,\n\nFor history buffs like me, computers are a dream come true!  Imagine having access to museums and libraries from all over the world, all at your fingertips. You can explore ancient civilizations, watch historical battles come to life in documentaries, or even take virtual tours of famous landmarks.\n\nTextbooks are cool, but computers let you experience history in a whole new way. You can see real photos and videos, listen to speeches from famous people, and even read primary source documents.  It's like stepping right back in time, without needing a time machine (although that would be pretty awesome too!).\n\nSo next time you hear someone complaining about computers, tell them about the history buff's paradise they're missing out on!\n\nSincerely,\n\nThe Time-Traveling Student (via Computer)",
        3
    ],
    [
        1,
        "Dear Editor,\n\nThe conversation about computers seems stuck in a binary – productivity boosters or social isolation machines. While I agree computers offer undeniable benefits – research efficiency, global communication, and even entertainment – I believe their impact on productivity might be a double-edged sword.\n\nOn the surface, computers streamline tasks, from research to collaboration. Gone are the days of scouring libraries; now, a quick keyword search yields mountains of information. However, this very convenience can be a productivity trap. The endless stream of notifications, social media updates, and the allure of online entertainment can easily derail focus.\n\nFurthermore, the multitasking culture computers foster might be an illusion. Studies suggest rapid switching between tasks actually diminishes efficiency. The constant barrage of stimuli creates a fear of missing out mentality, leading us to flit between tasks, achieving none in depth.\n\nSo, how do we utilize the benefits of computers without succumbing to their productivity paradox? Perhaps a conscious effort towards focused work sessions, utilizing time management tools, and even strategically disconnecting from the digital world could be the answer.\n\nComputers are powerful tools, but ultimately, it's up to us to harness their potential for true productivity, not get lost in the digital whirlwind.\n\nSincerely,\n\nA Student in the Age of Distraction",
        5
    ],
    [
        1,
        "Dear Editor,\n\nThe internet age, ushered in by computers, has undeniably expanded our access to information and connection. However, I believe the rise of \"echo chambers\" on social media platforms presents a significant threat to critical thinking and genuine discourse.\n\nComputers have created personalized online experiences. News feeds curate content based on our preferences, surrounding us with information that confirms our existing beliefs. This insular environment fosters confirmation bias, where we dismiss opposing viewpoints and cling to opinions that resonate with our own.\n\nThis lack of exposure to diverse perspectives hinders intellectual growth and stifles healthy debate.  Civil discourse necessitates grappling with opposing views, challenging our assumptions, and fostering empathy. Echo chambers, on the other hand, create intellectual bubbles, leading to polarization and a decline in open-mindedness.\n\nTo counteract these effects, we must consciously seek out diverse viewpoints on computers themselves. We can utilize search engines to find information from various sources and engage in respectful online discussions with those who hold differing opinions.\n\nComputers are powerful tools for connection, but they shouldn't limit our intellectual horizons.  Let's harness their potential to create a more informed and intellectually diverse society.\n\nSincerely,\n\nA Student Who Values Open Discourse",
        5
    ],
    [
        1,
        "Dear Editor,\n\nThe internet, facilitated by computers, has democratized access to information and ignited a wave of social activism. However, the ease of online engagement might be creating a culture of \"clicktivism\" – shallow, performative activism that prioritizes social media validation over meaningful action.\n\nComputers allow us to instantly share petitions, raise awareness for causes, and connect with like-minded individuals. However, this ease of engagement can lead to a sense of accomplishment without actual effort. Signing a petition online might feel good, but it doesn't translate to real-world change unless followed by concrete action.\n\nFurthermore, the curated nature of social media feeds can create a skewed perception of reality. We might be bombarded with success stories, fostering a sense of discouragement when our own efforts seem insignificant. This can lead to slacktivism, where individuals feel absolved of their responsibility after a simple click.\n\nComputers can be powerful tools for social change, but we must guard against mistaking online engagement for genuine activism. It's crucial to translate our digital outrage into concrete action, volunteering, and creating a dialogue with those who hold opposing views.  Only then can we  leverage the power of computers to create lasting social change.\n\nSincerely,\n\nA Student Who Believes in Action, Not Just Likes",
        5
    ],
    [
        1,
        "Dear Editor,\n\nComputers are like double-stuffed Oreos: awesome and delicious, but you gotta be careful not to overdo it. Sure, they can be educational – research projects are a breeze with all that info online, and some games are actually pretty challenging (think reflexes of a ninja!). Plus, you can video chat with your bestie across town, way cooler than a boring phone call.\n\nBut let's face it, computers can be a major time suck. You hop on to check something quick, and next thing you know, hours have flown by scrolling through social media or watching random videos.  Suddenly, you haven't gotten any homework done, and your mom's nagging you to get outside.\n\nThe key is balance, people! Computers are great tools, but they shouldn't replace real life. We need to get our exercise, spend time with friends and fam, and maybe even appreciate some fresh air (believe it or not, the world outside your screen is pretty cool too!). So, let's use computers to our advantage, but remember, there's a whole world out there waiting to be explored –  the non-digital kind!\n\nSincerely,\n\nThe Tech-Savvy (But Not Screen-Obsessed) Teenager",
        3
    ],
    [
        1,
        "Dear Editor,\n\nLook, I get it. Some adults think computers are turning us all into antisocial code monkeys. They whine about us glued to screens, neglecting exercise and real-life interactions. But here's the thing: computers can actually be social tools!\n\nThink about it. We can connect with friends online, share jokes and stories, even play games together virtually. Plus, social media lets you meet new people who share your interests, no matter how niche they are. It's like having a giant club for everything under the sun, all online!\n\nSure, computers shouldn't replace face-to-face interactions. But honestly, some people are just way more comfortable expressing themselves online. Plus, there are plenty of introverts and shy kids out there who benefit from the social connection computers offer.\n\nSo, next time you see a teenager staring at a screen, don't assume the worst. We might be having the best virtual hangout ever, making new friends, or even learning a new skill. Computers can be social butterflies too!\n\nYours truly,\n\nThe Socially Connected Screenager",
        3
    ],
    [
        1,
        "Dear Editor,\n\nComputers aren't just a trend, they're the future! They're changing the world, and it's not all doom and gloom like some grumpy grown-ups think.  Learning? Computers make it interactive and fun. Need to connect with someone across the globe? Done in seconds.  Plus, with online courses and coding programs, teenagers like me can actually learn future-proof skills.\n\nSure, too much screen time can be bad, but guess what? We're smart enough to find a balance.  We can use computers to learn about fitness and healthy habits. Plus, there are apps for tracking hikes and even virtual reality experiences that take you to exotic locations.\n\nHonestly, computers are opening doors to a whole new world. We can be part of the solution, not the problem. Let's embrace technology and use it to our advantage. The future is digital, and  guess what? We're ready for it!\n\nSincerely,\n\nThe Tech-Ready Teenager",
        3
    ],
    [
        1,
        "Dear Editor,\n\nThe rise of computers has sparked a heated debate – are they a boon or a bane for society? While some laud their educational and social advantages, concerns about human connection and physical well-being loom large.\n\nUndeniably, computers offer unparalleled access to information. From research for academic papers to virtual tours of the Colosseum, knowledge is democratized. Additionally, online platforms foster social connection – video calls bridge geographical distances, and online communities allow people with shared interests to connect.\n\nHowever, these benefits come at a cost. The allure of the digital world can foster social isolation. Face-to-face interactions, crucial for emotional well-being and developing social skills, can be neglected. Furthermore, the sedentary nature of computer use contributes to a decline in physical activity, leading to health problems.\n\nThe solution lies not in abandoning technology, but in cultivating a mindful relationship with it. We must prioritize real-world interactions and physical activities. Setting time limits for computer use and scheduling outdoor adventures are essential.\n\nUltimately, computers are tools – powerful and transformative, but ultimately neutral. It is up to us to harness their potential while remaining grounded in the real world. Let us not become slaves to technology, but rather its responsible stewards.\n\nSincerely,\n\nA Concerned Citizen of the Digital Age",
        5
    ],
    [
        1,
        "Dear Editor,\n\nThe pervasiveness of computers in our lives presents a double-edged sword – a vast ocean of information at our fingertips, coupled with the potential for intellectual stagnation. While proponents highlight the educational advantages of computers, the ease of access can breed a culture of passive information consumption.\n\nUndoubtedly, computers are gateways to knowledge. Research for projects becomes a breeze, and online courses offer unprecedented educational opportunities. Additionally, global news platforms allow us to stay informed about international affairs.\n\nHowever, the sheer volume of information available online can be overwhelming. The ability to discern credible sources from fake news is crucial, yet neglected. Furthermore, the reliance on pre-summarized information can hinder the development of critical thinking skills.\n\nTo mitigate these risks, we need to foster an environment that encourages active information processing. Evaluating sources, questioning assumptions, and drawing reasoned conclusions are paramount. Additionally, we must advocate for educational reforms that emphasize critical thinking alongside access to information.\n\nIn conclusion, computers can be powerful tools for learning, but only if we cultivate the skills to navigate the information landscape effectively. Let us not become passive consumers of data, but rather discerning learners who utilize technology to expand our intellectual horizons.\n\nSincerely,\n\nA Student of the Digital Age",
        5
    ],
    [
        1,
        "Dear Editor,\n\nThe rise of computers has coincided with a growing concern - the erosion of privacy in the digital age. While proponents highlight the social and educational benefits of computers, the trade-off is often our personal data becoming a valuable commodity for corporations.\n\nUndeniably, computers facilitate social interactions and educational pursuits. Online platforms connect us with loved ones across the globe, and online courses offer flexible learning opportunities.\n\nHowever, our online activities generate a vast trail of data – search history, social media interactions, even location tracking. This data is often collected and monetized by corporations, potentially leading to targeted advertising and even manipulation.\n\nThe solution lies in advocating for stricter data protection laws and fostering a culture of digital hygiene. We must be vigilant about the information we share online and utilize tools to protect our privacy. Additionally, promoting digital literacy is crucial to empower individuals to navigate the complexities of online data collection.\n\nIn conclusion, computers offer undeniable benefits, but at a cost – our privacy. Striking a balance between technological advancement and individual autonomy is paramount. Let us reclaim control over our digital footprints and ensure that computers are tools for empowerment, not instruments of surveillance.\n\nSincerely,\n\nA Concerned Citizen of the Digital Age",
        5
    ],
    [
        1,
        "Dear Editor,\n\nSome grown-ups might moan about computers turning our brains to mush, but they're totally wrong! Computers are like a giant learning playground, way cooler than dusty old textbooks. Wanna learn about the pyramids? Boom, there's a virtual tour online. Need help with a science project? There are tons of websites with experiments you can actually do at home (with adult supervision, of course!).\n\nPlus, forget boring lectures – some online courses have games and quizzes that make learning actually fun! And don't even get me started on coding. It's like learning a secret language that lets you build your own computer games!\n\nSure, sometimes you can get sucked into a black hole of cat videos, but that's why we need balance. Maybe parents can set screen time limits, and we can all make an effort to play outside more. But honestly, computers are here to stay, and they're making learning way more awesome!\n\nYour fellow student,\n\nThe Tech-Savvy Brainiac",
        3
    ],
    [
        1,
        "Dear Editor,\n\nComputers are awesome for some things, but they can also be a bit of a double-edged sword, especially when it comes to friends. On the one hand, they let you connect with people from all over the world, which is pretty cool. You can video chat with your cousin in Hawaii or chat with someone who shares your love for obscure cartoon characters.\n\nBut here's the thing: computers can't replace real-life friends. Sure, you can text your bestie all day, but there's nothing like hanging out in person, cracking jokes, and maybe even getting into a friendly game of dodgeball.\n\nSometimes, computers can make us forget about the people right next to us. We get so caught up in the online world that we forget to spend quality time with our family or even just play with the dog.\n\nSo, the key is balance. Let's use computers to connect with people far away, but don't forget to nurture the friendships we have in real life too. Remember, a computer friend can't give you a high five!\n\nSincerely,\n\nThe Socially Balanced Student",
        4
    ],
    [
        1,
        "Dear Editor,\n\nSome folks think computers are turning us into couch potatoes, glued to screens and forgetting about the great outdoors. They might have a point – who needs a walk in the park when you can explore a rainforest virtually?\n\nBut here's the secret: computers can actually help you appreciate nature even more! You can research cool animals you want to see someday, like majestic tigers or playful dolphins. Plus, there are apps that help you identify plants and track your hikes. Imagine hiking a mountain and using an app to learn all about the different trees you see!\n\nThink of computers as an escape hatch from the everyday. They can transport you to amazing places, but they can also inspire you to experience the real thing. So next time you're feeling cooped up, use your computer to plan an awesome outdoor adventure, not just another virtual one!\n\nYours truly,\n\nThe Tech-Powered Nature Enthusiast",
        4
    ],
    [
        1,
        "Dear Editor,\n\nThe rise of computers has fundamentally reshaped the landscape of knowledge acquisition. While concerns about attention spans and social isolation are valid, the democratization of information access outweighs these anxieties.\n\nPrior to the digital age, knowledge was largely siloed within academic institutions and specialized libraries. Today, computers provide unprecedented access to a vast ocean of information. Online databases, academic journals, and educational platforms empower individuals to pursue intellectual curiosity regardless of geographic location or socioeconomic background.\n\nHowever, the sheer volume of information can be overwhelming. Critical thinking skills are paramount in navigating the digital terrain, discerning credible sources from misinformation. Universities and educators must adapt, fostering information literacy alongside access.\n\nFurthermore, the always-on nature of technology can lead to social isolation and hinder physical well-being.  Encouraging digital  detoxification and promoting healthy screen time habits are crucial. Yet, the ability to connect with colleagues and friends globally fosters a sense of global citizenship and facilitates collaboration across borders.\n\nIn conclusion, the democratizing potential of computers outweighs concerns about their downsides. By fostering critical thinking skills and promoting balance in our digital lives, we can harness the power of technology to empower individuals and advance human knowledge.\n\nSincerely,\n\n",
        5
    ],
    [
        1,
        "Dear Editor,\n\nWhile computers hold immense potential for learning and connection, they can exacerbate existing societal inequalities.  While proponents tout the democratization of information, the reality is a widening algorithmic divide.\n\nUnequal access to computers and reliable internet connections limits opportunities for those in underserved communities.  Without this access, educational pursuits, professional development, and even basic communication are hindered.   Furthermore, the digital divide disproportionately affects marginalized groups, further perpetuating social and economic disparities.\n\nMoreover, the internet thrives on data, leading to algorithmic biases.  Search engines and social media platforms often prioritize content that confirms existing beliefs, creating echo chambers that hinder exposure to diverse viewpoints and critical thinking.\n\nTo mitigate these issues, we need robust investment in public libraries, affordable internet access programs, and digital literacy initiatives. Additionally, platforms need to be held accountable for algorithmic bias, ensuring inclusive and equitable information access.\n\nIn conclusion, the benefits of computers cannot be fully realized without addressing the digital divide and algorithmic bias.  Otherwise, these technological advancements will only exacerbate existing inequalities.  Let us bridge this gap and ensure that computers empower all individuals, not just a privileged few.\n\nSincerely,\n\nA Social Scientist of the Digital Age",
        5
    ],
]

In [6]:
# Generate essay IDs
start_id = 7085
essay_df = pd.DataFrame(data=data, columns=columns)
essay_df['essay_id'] = np.arange(start_id, start_id + len(data))

In [7]:
essay_df.head()

,essay_set,essay,ground_truth_score,essay_id
0,1,Is everyone freaking out about computers for n...,2,7085
1,1,"Dear Editor,\n\nComputers are like a double-ed...",4,7086
2,1,"Dear Editor,\n\nSome people say computers are ...",3,7087
3,1,"Dear Editor,\n\nComputers aren't just about ga...",4,7088
4,1,"Dear Editor,\n\nLet's be honest, computers can...",3,7089


In [8]:
essay_df.to_csv('finetune_set.csv', index=False)

In [16]:
class EssayDataset(Dataset):
    def __init__(self, finetune_set_path, tokenizer):
        self.tokenizer = tokenizer
        df = pd.read_csv(finetune_set_path)
        self.inputs = df['essay']
        self.labels = df['ground_truth_score']

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        input = self.inputs.iloc[idx]
        label = self.labels.iloc[idx]
        encodings = self.tokenizer(input,
                                   return_tensors='pt',
                                   truncation=True,
                                   padding='max_length',
                                   max_length=512)
        return encodings.input_ids.squeeze(), label

In [17]:
# Create the dataset
dataset = EssayDataset('finetune_set.csv', tokenizer)

### Finetuning the model

In [18]:
# Split the dataset into training and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [19]:
# Batch inputs
def data_collator(features):
    input_ids = torch.stack([f[0] for f in features])
    labels = torch.tensor([f[1] for f in features])
    labels = labels.view(-1, 1).expand_as(input_ids)
    return {'input_ids': input_ids, 'labels': labels}

In [20]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='epoch',
    save_strategy='epoch'
)

In [21]:
# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [22]:
# Fine-tune the model
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,12.521059
2,11.759300,12.057110
3,11.264300,11.322369


TrainOutput(global_step=24, training_loss=11.439710776011148, metrics={'train_runtime': 274.8156, 'train_samples_per_second': 0.175, 'train_steps_per_second': 0.087, 'total_flos': 12542017536000.0, 'train_loss': 11.439710776011148, 'epoch': 3.0})

### Save model

In [23]:
# Save the model
model.save_pretrained('./fine-tuned-gpt2')
tokenizer.save_pretrained('./fine-tuned-gpt2')

('./fine-tuned-gpt2\\tokenizer_config.json',
 './fine-tuned-gpt2\\special_tokens_map.json',
 './fine-tuned-gpt2\\vocab.json',
 './fine-tuned-gpt2\\merges.txt',
 './fine-tuned-gpt2\\added_tokens.json',
 './fine-tuned-gpt2\\tokenizer.json')

---

## Automated Essay Scoring

### Load test data

In [24]:
essay_df = pd.read_csv("data/training_set.tsv", sep='\t', encoding = "ISO-8859-1")
essay_df.head()

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,...,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Preprocessing

In [25]:
# Drop unused columns
essay_df = essay_df.drop(['rater2_domain1', 'domain1_score', 'rater3_domain1', 'rater1_domain2', 'rater2_domain2', 'domain2_score',
                          'rater1_trait1', 'rater1_trait2', 'rater1_trait3', 'rater1_trait4', 'rater1_trait5', 'rater1_trait6',
                          'rater2_trait1', 'rater2_trait2', 'rater2_trait3', 'rater2_trait4', 'rater2_trait5', 'rater2_trait6',
                          'rater3_trait1', 'rater3_trait2', 'rater3_trait3', 'rater3_trait4', 'rater3_trait5', 'rater3_trait6'],
                          axis=1)

In [26]:
# Drop unused essay sets
essay_df = essay_df[essay_df.essay_set == 1]

### Rubric Development

In [27]:
columns = ['essay_set', 'grade_level', 'prompt', 'rubrics', 'score_range']
data = [
    [
        1,
        8,
        "More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.",
        "Score Point 1: An undeveloped response that may take a position but offers no more than very minimal support. Typical elements: Contains few or vague details. Is awkward and fragmented. May be difficult to read and understand. May show no awareness of audience.\nScore Point 2: An under-developed response that may or may not take a position. Typical elements: Contains only general reasons with unelaborated and/or list-like details. Shows little or no evidence of organization. May be awkward and confused or simplistic. May show little awareness of audience. \nScore Point 3: A minimally-developed response that may take a position, but with inadequate support and details. Typical elements: Has reasons with minimal elaboration and more general than specific details. Shows some organization. May be awkward in parts with few transitions. Shows some awareness of audience. \nScore Point 4: A somewhat-developed response that takes a position and provides adequate support. Typical elements: Has adequately elaborated reasons with a mix of general and specific details. Shows satisfactory organization. May be somewhat fluent with some transitional language. Shows adequate awareness of audience. \nScore Point 5: A developed response that takes a clear position and provides reasonably persuasive support. Typical elements: Has moderately well elaborated reasons with mostly specific details. Exhibits generally strong organization. May be moderately fluent with transitional language throughout. May show a consistent awareness of audience. \nScore Point 6: A well-developed response that takes a clear and thoughtful position and provides persuasive support. Typical elements: Has fully elaborated reasons with specific details. Exhibits strong organization. Is fluent and uses sophisticated transitional language. May show a heightened awareness of audience.",
        "1-6"
    ],
    [
        3,
        10,
        "Source Essay: ROUGH ROAD AHEAD: Do Not Exceed Posted Speed Limit by Joe Kurmaskie FORGET THAT OLD SAYING ABOUT NEVER taking candy from strangers. No, a better piece of advice for the solo cyclist would be, “Never accept travel advice from a collection of old-timers who haven’t left the confines of their porches since Carter was in office.” It’s not that a group of old guys doesn’t know the terrain. With age comes wisdom and all that, but the world is a fluid place. Things change. At a reservoir campground outside of Lodi, California, I enjoyed the serenity of an early-summer evening and some lively conversation with these old codgers. What I shouldn’t have done was let them have a peek at my map. Like a foolish youth, the next morning I followed their advice and launched out at first light along a “shortcut” that was to slice away hours from my ride to Yosemite National Park. They’d sounded so sure of themselves when pointing out landmarks and spouting off towns I would come to along this breezy jaunt. Things began well enough. I rode into the morning with strong legs and a smile on my face. About forty miles into the pedal, I arrived at the first “town.” This place might have been a thriving little spot at one time—say, before the last world war—but on that morning it fit the traditional definition of a ghost town. I chuckled, checked my water supply, and moved on. The sun was beginning to beat down, but I barely noticed it. The cool pines and rushing rivers of Yosemite had my name written all over them. Twenty miles up the road, I came to a fork of sorts. One ramshackle shed, several rusty pumps, and a corral that couldn’t hold in the lamest mule greeted me. This sight was troubling. I had been hitting my water bottles pretty regularly, and I was traveling through the high deserts of California in June. I got down on my hands and knees, working the handle of the rusted water pump with all my strength. A tarlike substance oozed out, followed by brackish water feeling somewhere in the neighborhood of two hundred degrees. I pumped that handle for several minutes, but the water wouldn’t cool down. It didn’t matter. When I tried a drop or two, it had the flavor of battery acid. The old guys had sworn the next town was only eighteen miles down the road. I could make that! I would conserve my water and go inward for an hour or so—a test of my inner spirit. Not two miles into this next section of the ride, I noticed the terrain changing. Flat road was replaced by short, rolling hills. After I had crested the first few of these, a large highway sign jumped out at me. It read: ROUGH ROAD AHEAD: DO NOT EXCEED POSTED SPEED LIMIT. The speed limit was 55 mph. I was doing a water-depleting 12 mph. Sometimes life can feel so cruel. I toiled on. At some point, tumbleweeds crossed my path and a ridiculously large snake—it really did look like a diamondback—blocked the majority of the pavement in front of me. I eased past, trying to keep my balance in my dehydrated state. The water bottles contained only a few tantalizing sips. Wide rings of dried sweat circled my shirt, and the growing realization that I could drop from heatstroke on a gorgeous day in June simply because I listened to some gentlemen who hadn’t been off their porch in decades, caused me to laugh. It was a sad, hopeless laugh, mind you, but at least I still had the energy to feel sorry for myself. There was no one in sight, not a building, car, or structure of any kind. I began breaking the ride down into distances I could see on the horizon, telling myself that if I could make it that far, I’d be fi ne. Over one long, crippling hill, a building came into view. I wiped the sweat from my eyes to make sure it wasn’t a mirage, and tried not to get too excited. With what I believed was my last burst of energy, I maneuvered down the hill. In an ironic twist that should please all sadists reading this, the building—abandoned years earlier, by the looks of it—had been a Welch’s Grape Juice factory and bottling plant. A sandblasted picture of a young boy pouring a refreshing glass of juice into his mouth could still be seen. I hung my head. That smoky blues tune “Summertime” rattled around in the dry honeycombs of my deteriorating brain. I got back on the bike, but not before I gathered up a few pebbles and stuck them in my mouth. I’d read once that sucking on stones helps take your mind off thirst by allowing what spit you have left to circulate. With any luck I’d hit a bump and lodge one in my throat. It didn’t really matter. I was going to die and the birds would pick me clean, leaving only some expensive outdoor gear and a diary with the last entry in praise of old men, their wisdom, and their keen sense of direction. I made a mental note to change that paragraph if it looked like I was going to lose consciousness for the last time. Somehow, I climbed away from the abandoned factory of juices and dreams, slowly gaining elevation while losing hope. Then, as easily as rounding a bend, my troubles, thirst, and fear were all behind me. GARY AND WILBER’S FISH CAMP—IF YOU WANT BAIT FOR THE BIG ONES, WE’RE YOUR BEST BET! “And the only bet,” I remember thinking. As I stumbled into a rather modern bathroom and drank deeply from the sink, I had an overwhelming urge to seek out Gary and Wilber, kiss them, and buy some bait—any bait, even though I didn’t own a rod or reel. An old guy sitting in a chair under some shade nodded in my direction. Cool water dripped from my head as I slumped against the wall beside him. “Where you headed in such a hurry?” “Yosemite,” I whispered. “Know the best way to get there?” I watched him from the corner of my eye for a long moment. He was even older than the group I’d listened to in Lodi. “Yes, sir! I own a very good map.” And I promised myself right then that I’d always stick to it in the future. “Rough Road Ahead” by Joe Kurmaskie, from Metal Cowboy, copyright © 1999 Joe Kurmaskie. Instructions: Write a response that explains how the features of the setting affect the cyclist. In your response, include examples from the essay that support your conclusion.",
        "Score Point 3: The response demonstrates an understanding of the complexities of the text. Addresses the demands of the question Uses expressed and implied information from the text Clarifies and extends understanding beyond the literal \nScore 2: The response demonstrates a partial or literal understanding of the text. Addresses the demands of the question, although may not develop all parts equally Uses some expressed or implied information from the text to demonstrate understanding May not fully connect the support to a conclusion or assertion made about the text(s) \nScore 1: The response shows evidence of a minimal understanding of the text. May show evidence that some meaning has been derived from the text May indicate a misreading of the text or the question May lack information or explanation to support an understanding of the text in relation to the question \nScore Point 0: The response is completely irrelevant or incorrect, or there is no response.",
        "0-3"
    ],
    [
        4,
        10,
        'Source Essay: Winter Hibiscus by Minfong Ho Saeng, a teenage girl, and her family have moved to the United States from Vietnam. As Saeng walks home after failing her driver’s test, she sees a familiar plant. Later, she goes to a florist shop to see if the plant can be purchased. It was like walking into another world. A hot, moist world exploding with greenery. Huge flat leaves, delicate wisps of tendrils, ferns and fronds and vines of all shades and shapes grew in seemingly random profusion. “Over there, in the corner, the hibiscus. Is that what you mean?” The florist pointed at a leafy potted plant by the corner. There, in a shaft of the wan afternoon sunlight, was a single blood-red blossom, its five petals splayed back to reveal a long stamen tipped with yellow pollen. Saeng felt a shock of recognition so intense, it was almost visceral.1 “Saebba,” Saeng whispered. A saebba hedge, tall and lush, had surrounded their garden, its lush green leaves dotted with vermilion flowers. And sometimes after a monsoon rain, a blossom or two would have blown into the well, so that when she drew the well water, she would find a red blossom floating in the bucket. Slowly, Saeng walked down the narrow aisle toward the hibiscus. Orchids, lanna bushes, oleanders, elephant ear begonias, and bougainvillea vines surrounded her. Plants that she had not even realized she had known but had forgotten drew her back into her childhood world. When she got to the hibiscus, she reached out and touched a petal gently. It felt smooth and cool, with a hint of velvet toward the center—just as she had known it would feel. And beside it was yet another old friend, a small shrub with waxy leaves and dainty flowers with purplish petals and white centers. “Madagascar periwinkle,” its tag announced. How strange to see it in a pot, Saeng thought. Back home it just grew wild, jutting out from the cracks in brick walls or between tiled roofs. And that rich, sweet scent—that was familiar, too. Saeng scanned the greenery around her and found a tall, gangly plant with exquisite little white blossoms on it. “Dok Malik,” she said, savoring the feel of the word on her tongue, even as she silently noted the English name on its tag, “jasmine.” One of the blossoms had fallen off, and carefully Saeng picked it up and smelled it. She closed her eyes and breathed in, deeply. The familiar fragrance filled her lungs, and Saeng could almost feel the light strands of her grandmother’s long gray hair, freshly washed, as she combed it out with the fine-toothed buffalo-horn comb. And when the sun had dried it, Saeng would help the gnarled old fingers knot the hair into a bun, then slip a dok Malik bud into it. Saeng looked at the white bud in her hand now, small and fragile. Gently, she closed her palm around it and held it tight. That, at least, she could hold on to. But where was the fine-toothed comb? The hibiscus hedge? The well? Her gentle grandmother? A wave of loss so deep and strong that it stung Saeng’s eyes now swept over her. A blink, a channel switch, a boat ride into the night, and it was all gone. Irretrievably, irrevocably gone. And in the warm moist shelter of the greenhouse, Saeng broke down and wept. It was already dusk when Saeng reached home. The wind was blowing harder, tearing off the last remnants of green in the chicory weeds that were growing out of the cracks in the sidewalk. As if oblivious to the cold, her mother was still out in the vegetable garden, digging up the last of the onions with a rusty trowel. She did not see Saeng until the girl had quietly knelt down next to her. Her smile of welcome warmed Saeng. “Ghup ma laio le? You’re back?” she said cheerfully. “Goodness, it’s past five. What took you so long? How did it go? Did you—?” Then she noticed the potted plant that Saeng was holding, its leaves quivering in the wind. Mrs. Panouvong uttered a small cry of surprise and delight. “Dok faeng-noi!” she said. “Where did you get it?” “I bought it,” Saeng answered, dreading her mother’s next question. “How much?” For answer Saeng handed her mother some coins. “That’s all?” Mrs. Panouvong said, appalled, “Oh, but I forgot! You and the Lambert boy ate Bee-Maags . . . .” “No, we didn’t, Mother,” Saeng said. “Then what else—?” “Nothing else. I paid over nineteen dollars for it.” “You what?” Her mother stared at her incredulously. “But how could you? All the seeds for this vegetable garden didn’t cost that much! You know how much we—” She paused, as she noticed the tearstains on her daughter’s cheeks and her puffy eyes. “What happened?” she asked, more gently. “I—I failed the test,” Saeng said. For a long moment Mrs. Panouvong said nothing. Saeng did not dare look her mother in the eye. Instead, she stared at the hibiscus plant and nervously tore off a leaf, shredding it to bits. Her mother reached out and brushed the fragments of green off Saeng’s hands. “It’s a beautiful plant, this dok faeng-noi,” she finally said. “I’m glad you got it.” “It’s—it’s not a real one,” Saeng mumbled. “I mean, not like the kind we had at—at—” She found that she was still too shaky to say the words at home, lest she burst into tears again. “Not like the kind we had before,” she said. “I know,” her mother said quietly. “I’ve seen this kind blooming along the lake. Its flowers aren’t as pretty, but it’s strong enough to make it through the cold months here, this winter hibiscus. That’s what matters.” She tipped the pot and deftly eased the ball of soil out, balancing the rest of the plant in her other hand. “Look how root-bound it is, poor thing,” she said. “Let’s plant it, right now.” She went over to the corner of the vegetable patch and started to dig a hole in the ground. The soil was cold and hard, and she had trouble thrusting the shovel into it. Wisps of her gray hair trailed out in the breeze, and her slight frown deepened the wrinkles around her eyes. There was a frail, wiry beauty to her that touched Saeng deeply. “Here, let me help, Mother,” she offered, getting up and taking the shovel away from her. Mrs. Panouvong made no resistance. “I’ll bring in the hot peppers and bitter melons, then, and start dinner. How would you like an omelet with slices of the bitter melon?” “I’d love it,” Saeng said. Left alone in the garden, Saeng dug out a hole and carefully lowered the “winter hibiscus” into it. She could hear the sounds of cooking from the kitchen now, the beating of eggs against a bowl, the sizzle of hot oil in the pan. The pungent smell of bitter melon wafted out, and Saeng’s mouth watered. It was a cultivated taste, she had discovered—none of her classmates or friends, not even Mrs. Lambert, liked it—this sharp, bitter melon that left a golden aftertaste on the tongue. But she had grown up eating it and, she admitted to herself, much preferred it to a Big Mac. The “winter hibiscus” was in the ground now, and Saeng tamped down the soil around it. Overhead, a flock of Canada geese flew by, their faint honks clear and—yes—familiar to Saeng now. Almost reluctantly, she realized that many of the things that she had thought of as strange before had become, through the quiet repetition of season upon season, almost familiar to her now. Like the geese. She lifted her head and watched as their distinctive V was etched against the evening sky, slowly fading into the distance. When they come back, Saeng vowed silently to herself, in the spring, when the snows melt and the geese return and this hibiscus is budding, then I will take that test again. “Winter Hibiscus” by Minfong Ho, copyright © 1993 by Minfong Ho, from Join In, Multiethnic Short Stories, by Donald R. Gallo, ed. Instructions: Read the last paragraph of the source essay given earlier. "When they come back, Saeng vowed silently to herself, in the spring, when the snows melt and the geese return and this hibiscus is budding, then I will take that test again." Write a response that explains why the author concludes the story with this paragraph. In your response, include details and examples from the story that support your ideas.',
        "Score Point 3: The response demonstrates an understanding of the complexities of the text. Addresses the demands of the question Uses expressed and implied information from the text Clarifies and extends understanding beyond the literal \nScore 2: The response demonstrates a partial or literal understanding of the text. Addresses the demands of the question, although may not develop all parts equally Uses some expressed or implied information from the text to demonstrate understanding May not fully connect the support to a conclusion or assertion made about the text(s) \nScore 1: The response shows evidence of a minimal understanding of the text. May show evidence that some meaning has been derived from the text May indicate a misreading of the text or the question May lack information or explanation to support an understanding of the text in relation to the question \nScore Point 0: The response is completely irrelevant or incorrect, or there is no response.",
        "0-3"
    ],
    [
        5,
        8,
        "Source Essay: Narciso Rodriguez from Home: The Blueprints of Our Lives My parents, originally from Cuba, arrived in the United States in 1956. After living for a year in a furnished one-room apartment, twenty-one-year-old Rawedia Maria and twenty-seven-year-old Narciso Rodriguez, Sr., could afford to move into a modest, three-room apartment I would soon call home. In 1961, I was born into this simple house, situated in a two-family, blond-brick building in the Ironbound section of Newark, New Jersey. Within its walls, my young parents created our traditional Cuban home, the very heart of which was the kitchen. My parents both shared cooking duties and unwittingly passed on to me their rich culinary skills and a love of cooking that is still with me today (and for which I am eternally grateful). Passionate Cuban music (which I adore to this day) filled the air, mixing with the aromas of the kitchen. Here, the innocence of childhood, the congregation of family and friends, and endless celebrations that encompassed both, formed the backdrop to life in our warm home. Growing up in this environment instilled in me a great sense that “family” had nothing to do with being a blood relative. Quite the contrary, our neighborhood was made up of mostly Spanish, Cuban, and Italian immigrants at a time when overt racism was the norm and segregation prevailed in the United States. In our neighborhood, despite customs elsewhere, all of these cultures came together in great solidarity and friendship. It was a close-knit community of honest, hardworking immigrants who extended a hand to people who, while not necessarily their own kind, were clearly in need. Our landlord and his daughter, Alegria (my babysitter and first friend), lived above us, and Alegria graced our kitchen table for meals more often than not. Also at the table were Sergio and Edelmira, my surrogate grandparents who lived in the basement apartment. (I would not know my “real” grandparents, Narciso the Elder and Consuelo, until 1970 when they were allowed to leave Cuba.) My aunts Bertha and Juanita and my cousins Arnold, Maria, and Rosemary also all lived nearby and regularly joined us at our table. Countless extended family members came and went — and there was often someone staying with us temporarily until they were able to get back on their feet. My parents always kept their arms and their door open to the many people we considered family, knowing that they would do the same for us. My mother and father had come to this country with such courage, without any knowledge of the language or the culture. They came selflessly, as many immigrants do, to give their children a better life, even though it meant leaving behind their families, friends, and careers in the country they loved. They struggled both personally and financially, braving the harsh northern winters while yearning for their native tropics and facing cultural hardships. The barriers to work were strong and high, and my parents both had to accept that they might not be able to find the kind of jobs they deserved. In Cuba, Narciso, Sr., had worked in a laboratory and Rawedia Maria had studied chemical engineering. In the United States, they had to start their lives over entirely, taking whatever work they could find. The faith that this struggle would lead them and their children to better times drove them to endure these hard times. I will always be grateful to my parents for their love and sacrifice. I’ve often told them that what they did was a much more courageous thing than I could have ever done. I’ve often told them of my admiration for their strength and perseverance, and I’ve thanked them repeatedly. But, in reality, there is no way to express my gratitude for the spirit of generosity impressed upon me at such an early age and the demonstration of how important family and friends are. These are two lessons that my parents did not just tell me. They showed me with their lives, and these teachings have been the basis of my life. It was in this simple house that my parents welcomed other refugees to celebrate their arrival to this country and where I celebrated my first birthdays. It was in the warmth of the kitchen in this humble house where a Cuban feast (albeit a frugal Cuban feast) always filled the air with not just scent and music but life and love. It was here where I learned the real definition of “family.” And for this, I will never forget that house or its gracious neighborhood or the many things I learned there about how to love. I will never forget how my parents turned this simple house into a home. — Narciso Rodriguez, Fashion designer Hometown: Newark, New Jersey “Narciso Rodriguez” by Narciso Rodriguez, from Home: The Blueprints of Our Lives. Copyright © 2006 by John Edwards. Instructions: Describe the mood created by the author in the memoir. Support your answer with relevant and specific information from the memoir.",
        "Score Point 4: The response is a clear, complete, and accurate description of the mood created by the author. The response includes relevant and specific information from the memoir. \nScore Point 3: The response is a mostly clear, complete, and accurate description of the mood created by the author. The response includes relevant but often general information from the memoir. \nScore Point 2: The response is a partial description of the mood created by the author. The response includes limited information from the memoir and may include misinterpretations. \nScore Point 1: The response is a minimal description of the mood created by the author. The response includes little or no information from the memoir and may include misinterpretations. OR The response relates minimally to the task. \nScore Point 0: The response is incorrect or irrelevant or contains insufficient information to demonstrate comprehension.",
        "0-4"
    ]
]

essay_rubric_df = pd.DataFrame(data=data, columns=columns)
essay_rubric_df.head()

,essay_set,grade_level,prompt,rubrics,score_range
0,1,8,"More and more people use computers, but not ev...",Score Point 1: An undeveloped response that ma...,1-6
1,3,10,Source Essay: ROUGH ROAD AHEAD: Do Not Exceed ...,Score Point 3: The response demonstrates an un...,0-3
2,4,10,Source Essay: Winter Hibiscus by Minfong Ho Sa...,Score Point 3: The response demonstrates an un...,0-3
3,5,8,Source Essay: Narciso Rodriguez from Home: The...,"Score Point 4: The response is a clear, comple...",0-4


### Model Prompting

In [28]:
def create_essay_eval_prompt(current_rubric, essay):
    prompt = "You are tasked to grade an essay written by a grade <grade_level> student given a set of rubrics, an essay prompt, and a range of possible scores to give. These are as follows:\n\nEssay prompt:\n<prompt>\n\nRubrics:\n<rubrics>\n\nEssay:\n<submission>\n\nDesired Output:\nOutput a single numerical grade between the range of <score_range>, inclusive, to grade this essay. In addition, give a brief explanation for the given grade (<400 words). Output this in the following format:\nScore: <score>\nExplanation: <explanation>"
    
    prompt = prompt.replace("<grade_level>", str(current_rubric.grade_level))
    prompt = prompt.replace("<prompt>", current_rubric.prompt)
    prompt = prompt.replace("<rubrics>", current_rubric.rubrics)
    prompt = prompt.replace("<submission>", essay)
    prompt = prompt.replace("<score_range>", current_rubric.score_range)
    
    return prompt

In [29]:
# Test prompt generation
test_essay = "Is everyone freaking out about computers for no reason? Computers are awesome! They're like having a superhero sidekick in your pocket. They can teach you anything you want to know, from how to build a volcano (for science class, of course!) to what life is like on the other side of the world. Plus, games help your hand-eye coordination, which is basically a superpower for gamers like me.\n\nSure, maybe some grown-ups spend too much time staring at screens, but that's their problem, not ours. We can totally use computers for good stuff and still hang out with friends or play outside.  So, lighten up everyone, computers are here to stay, and they're making our lives way cooler!\n\nYour friend,\n\nGamer Girl extraordinaire"
test_prompt = create_essay_eval_prompt(essay_rubric_df[essay_rubric_df.essay_set == 1].T.iloc[:,0], test_essay)
print(test_prompt)

You are tasked to grade an essay written by a grade 8 student given a set of rubrics, an essay prompt, and a range of possible scores to give. These are as follows:

Essay prompt:
More and more people use computers, but not everyone agrees that this benefits society. Those who support advances in technology believe that computers have a positive effect on people. They teach hand-eye coordination, give people the ability to learn about faraway places and people, and even allow people to talk online with other people. Others have different ideas. Some experts are concerned that people are spending too much time on their computers and less time exercising, enjoying nature, and interacting with family and friends. Write a letter to your local newspaper in which you state your opinion on the effects computers have on people. Persuade the readers to agree with you.

Rubrics:
Score Point 1: An undeveloped response that may take a position but offers no more than very minimal support. Typical 

### Model Evaluation

In [36]:
def get_essay_eval(essay_set, essay):
    current_rubric = essay_rubric_df[essay_rubric_df.essay_set == essay_set].T.iloc[:,0]
    essay_train_prompt = create_essay_eval_prompt(current_rubric, essay)
    input_tokens = tokenizer(essay_train_prompt, return_tensors="pt").to(device)
    
    with torch.no_grad(): # Tells PyTorch not to train the model on your input
        output_tokens = model.generate(input_tokens ["input_ids"], num_return_sequences=1, max_length=1050)

    output = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return output

In [37]:
essay_results_df = pd.DataFrame()

for index, row in essay_df.iterrows():
    eval = get_essay_eval(row.essay_set, row.essay)

    score = eval[len('Score: '): eval.find('\n')]
    explanation = eval[(eval.find('Explanation: ') + len('Explanation: ')):]

    result = {'essay_id': row.essay_id,
              'model_score': int(score),
              'explanation': explanation,
              'ground_truth_score': row.rater1_domain1}

    essay_results_df = pd.concat([essay_results_df, pd.DataFrame([result])], ignore_index=True)
    print(f'Evaluating essay {index+1}/{len(essay_df)}')

    # for testing
    if (index == 2):
      break

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


IndexError: index out of range in self

In [ ]:
essay_results_df.head()

In [ ]:
essay_results_df.describe()

In [ ]:
essay_qwk = cohen_kappa_score(essay_results_df.model_score, essay_results_df.ground_truth_score)
print(f"QWK score for essay autograding: {essay_qwk}")